## Implementing r200b scaling
The following cells build utilities and computations to develop on the idea presented in the latest update of 25th May in the introduction.<br>
r200b will be found from M200b and the background density.

## linDensityLOS(i,d)
The statistics developed above for LOS analysis have a redundancy due to direction. The density field on either side of a halo of interest should not be treated separately if we believe in the isotropy of the Universe. Hence, a still better estimate should be to analyse the density of the distribution around the halo of interest. This will become clear through the description of the following procedure-
<ol>
    <li>Lines of Sight are shot parallel to the grid through the centers of all the Halos of interest from a given mock catalogue and the overdensity amplitude is computed at every scale around the halo.</li>
    <li>A bracket of <i>L MPc</i> is introduced symmetrically around the Halo of interest. The average linear density of total amplitude enclosed in the bracket is found and plotted against scale. In the formula below <i>i</i> iterates over all grid points contained in the enclosing bracket imposed on the Halo environment.
        $$\lambda(L)=\dfrac{\sum_{i}A_i}{L}$$

</ol>

In [1]:
# This cell: Libraries imported and general data to be used is loaded
import sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time

colors=['g','b','r']
sourcePath="../SourceData/"  # Will not become a member of the git commits because CIC file is 1.1GB
fileNames=['b1','b1alpha','b1T10']
# Loading CIC data of density field. (512^3)
GridSize=512  # cubed of course.
cicSource="cic_snap049_grid512.dat"
data=np.reshape(np.fromfile(sourcePath+cicSource),(GridSize,GridSize,GridSize))

In [2]:
# This cell: Cosmology is defined. Simulation details are listed. Other constants to be used throughout are specified.
# Aseem's Cosmology
sigma_8=0.811
ns=0.961
h=0.7
Ob=0.045
Om=0.276

z=2.3         # Redshift specification
M_sun=1.989e+30  # M_sun in kg
Mega_parsec=3.086e+22  # parsec in metres
Delta=200.0   # Overdensity definition = Delta X background
rho_cr=((3*(100*h)**2)/(8*np.pi*6.673e-11))*((Mega_parsec/h**3)*1e+6/(M_sun/h))
# critical density of the Universe today 3H^2/8Pi*G in units M_sun.h^-1/(MPc.h^-1)^3
rho_m=Om*rho_cr # Units same as rho_cr. Don't need redshift considerations in comoving units i.e. Msun/h and MPc/h
del_crit=1.69
Lbox=150.0   # MPc/h

In [3]:
# Loading master catalogue data and putting it in usbale format
masterCat=np.loadtxt(sourcePath+"out_49.trees")

treesID=np.array(masterCat[:,1],dtype=int)  # ID of halos in the master catalogue
treesR200b=np.cbrt((masterCat[:,36]*3)/(4*np.pi*rho_m*200))   # The r200b of each halo from the master catalogue.
treesMap=np.column_stack((treesID,treesR200b))
treesMap=treesMap[treesMap[:,0].argsort()]   # TreesMap is a 2 column array with R200b for given ID of the halo
# TreesMap is sorted wrt the ID column so that Binary Search can be done on it too save shitloads of time.

In [4]:
# Loading the given Halo catalogues from original non-pickled files to get ID's and positions
b1=np.loadtxt(sourcePath+'b1.txt')
b1alpha=np.loadtxt(sourcePath+'b1alpha.txt')
b1T10=np.loadtxt(sourcePath+'b1T10.txt')

b1ID,b1alphaID,b1T10ID=np.array(b1[:,0],dtype=int),np.array(b1alpha[:,0],dtype=int),np.array(b1T10[:,0],dtype=int)
b1Pos,b1alphaPos,b1T10Pos=[],[],[]

b1Pos.append(b1[:,1])
b1Pos.append(b1[:,2])
b1Pos.append(b1[:,3])
b1Pos=np.array(b1Pos)
b1Pos=b1Pos.T  # Holds the positions of all the b1 halos in order according to b1T10

b1alphaPos.append(b1alpha[:,1])
b1alphaPos.append(b1alpha[:,2])
b1alphaPos.append(b1alpha[:,3])
b1alphaPos=np.array(b1alphaPos)
b1alphaPos=b1alphaPos.T

b1T10Pos.append(b1T10[:,1])
b1T10Pos.append(b1T10[:,2])
b1T10Pos.append(b1T10[:,3])
b1T10Pos=np.array(b1T10Pos)
b1T10Pos=b1T10Pos.T

In [5]:
# Returns an array of r200b for each ID passed as an argument, in order
# Note that treesMap is sorted on IDs and hence, we can run a Binary Search on it

def findR200b(ID):
    low,high=0,treesMap[:,0].size-1
    while(low<=high):
        mid=int(low+(high-low)/2)
        if(ID==treesMap[mid,0]):
            return(treesMap[mid,1])
        elif(ID>treesMap[mid,0]):
            low=mid+1
        else:
            high=mid-1
    return(-1)

In [6]:
# consistency check cell. Please ignore.
t10r,alphar,b1r=[],[],[]
for i in b1ID:
    b1r.append(findR200b(i))
for j in b1alphaID:
    alphar.append(findR200b(j))
for k in b1T10ID:
    t10r.append(findR200b(k))
    
b1r=np.array(b1r)
alphar=np.array(alphar)
t10r=np.array(t10r)
print(np.array([(np.mean(10*b1r)),np.mean(10*alphar),np.mean(10*t10r)]))

[1.00645603 0.89497071 1.81024704]


In [7]:
# Helping functions for asymmetric STD calculations in averagedLOS()
# Asymmetry in statistics. Data is split into values above and below mean. A different standard deviation is found for each set.

# This function finds the STD given the mean value
def std_dev(a,mean):
    if len(a)==0:
        return 0
    else:
        return np.sqrt(np.sum((a-mean)**2)/a.size)

In [8]:


def customStats(pos,DF,mode):
    allModes=np.array(['bin','interp'])  # list of all available modes. Add to this list if developed further
    if not any(mode==allModes):
        print("Please enter a valid mode. Current available modes are - "+str(allModes))
        print("After entering a valid mode, run again.")
        return
    
    

In [ ]:
# This cell tests the code above
p=np.array([1.,2.,3.,4.,5.,6.,7.])
d=np.array([np.array([1.,2.,3.]),np.array([1.,2.,3.,4.,5.]),np.array([1.]),np.array([1.,2.,3.,4.,5.,6.,7.])],)
#res,su,sd=customStats(p,d,'percentile')
#print(res,su,sd)


In [9]:
# The LOS Method implemented has lines always passing parallel to x-axis and through the grid points
# Direction specifies which direction the LOS are parallel to
# r_lim specifies the limit to which LOS calculations should be done about the central position of the halo
# Limit is specified as distance in the real position coordinates
# Also presenting the linear density around the halo of interest

def simpleR200bLOS(halo,r200b,direction='x'):
    pos,df=np.array([]),np.array([])
    rlim=10*r200b  # the upper limit to which Density Field should be found
    
    grid2box=Lbox/GridSize   # MPc/CICcell
    box2grid=(GridSize-1)/Lbox   # CICcells/MPc
    
    # Connvert Halo co-odinates from real space to closest CIC Grid positions
    X=int(np.floor(halo[0]*box2grid))
    Y=int(np.floor(halo[1]*box2grid))
    Z=int(np.floor(halo[2]*box2grid))
    
    # Initial values directly appended
    pos=np.append(pos,0.0)
    df=np.append(df,data[X,Y,Z])
    
    # Appending to front and back of the DF array iteratively, starting from center to make sure center is def included
    scale_grid=1
    scale=scale_grid*grid2box
    while scale<=rlim:   # runs till real scale is less than  rlim. NOt we started from -rlim.
        pos=np.append(pos,scale/r200b)
        pos=np.concatenate(([-scale/r200b],pos))
        
        pr,pl=[],[]
        if direction=='x':
            pr=np.array([X+scale_grid,Y,Z])
            pl=np.array([X-scale_grid,Y,Z])
        elif direction=='y':
            pr=np.array([X,Y+scale_grid,Z])
            pl=np.array([X,Y-scale_grid,Z])
        elif direction=='z':
            pr=np.array([X,Y,Z+scale_grid])   
            pl=np.array([X,Y,Z-scale_grid])   
        else:
            print("Please choose a valid direction for the LOS. Valid choices are - x | y | z")
            exit()
            
        pr[pr>=GridSize]=pr[pr>=GridSize]-GridSize
        pr[pr<0]=pr[pr<0]+GridSize   # periodicty of the box considered
        df=np.append(df,data[pr[0],pr[1],pr[2]])
        
        pl[pl>=GridSize]=pl[pl>=GridSize]-GridSize
        pl[pl<0]=pl[pl<0]+GridSize   # periodicty of the box considered
        
        df=np.concatenate(([data[pl[0],pl[1],pl[2]]],df))
        
        scale_grid+=1
        scale=scale_grid*grid2box
        
    return np.array(pos),np.array(df)

In [10]:
# LOS method implemented has lines passing through every halo of each catalogue to give a statistically robust estimate
# Arguments to be passed are halo IDs and corresponding positions...
# ...how many directions to consider for calculating the LOS - all | x | y | z

def averagedLOS_R200b(ID,pos,los_direction='all'):
    
    options=np.array(['all','x','y','z'])
    if not any(options == los_direction):
        print("Please enter a valid value to direction argument. Valid values -- all | x | y | z")
        print("Jupyter Notebook kernel will be killed")
        exit()
    
    mocks=pos
    masterPos,masterDF=[],[]
    # masterDF will store LOS DF for each halo in mocks
    # resultDF will be the mean of masterDF at each scale
    
    if los_direction == 'all':
        for h in range(0,mocks[:,0].size):
            for directn in ['x','y','z']:
                scales,df=simpleR200bLOS(mocks[h],findR200b(ID[h]),directn)
                masterDF.append(df)
                masterPos.append(scales)
        
    else:
        for h in range(0,mocks[:,0].size):
            scales,df=simpleR200bLOS(mocks[h],findR200b(ID[h]),los_direction)
            masterDF.append(df)
            masterPos.append(scales)
    
    # ---------------------------------------------------------------------------------------------------------------
    # No 2 halos have the same R200b values, hence, all the DF values at R/R200b scales are found at different...
    #... non-matching scales. Due to this statistics can't be run on these points because we don't have DF values at...
    #... a single R/R200b.
    # 2 solutions to these are to either make bins or do an interpolation
    # Issue with bins is that the mean spacing of R/R200b between two neighboring points is too large and we can fit...
    #... only about 3-4 bins on one side of the Halo of interest.
    # Issue with interpolation is that the number of points to interpolate between is also very small, evidently,...
    #... the interpolation is not expected to be very robust
    # ---------------------------------------------------------------------------------------------------------------
    
    '''
    resultDF,std_up,std_below=customStats(masterPos,masterDF,mode='std') # mode='std' | mode='percentile'
    #resultDF,prcntile16,prcntile84=customStats(masterPos,masterDF,mode='percentile')
    # 'percentile' mode recommended because 'std' mode might return 0 for stan. dev. due to lack of data points
    
    if len(masterPos)==len(resultDF):
        return masterPos,resultDF,std_up,std_below
        #return masterPos,resultDF,prcntile16,prcntile84
    else:
        print("Fatal error. Pos and DF sizes don't match.")
        return
    '''
    
    return masterPos,masterDF

In [ ]:
p,d=averagedLOS_R200b(b1T10ID,b1T10Pos,'x')

In [ ]:
mi,ma=p[0][0],p[0][-1]
mi_in,ma_in=0,0
i=0
for a in p:
    if a[0]<mi:
        mi=a[0]
        mi_in=i
    if a[-1]>ma:
        ma=a[-1]
        ma_in=i
    i+=1
print(mi,ma)
print(mi_in,ma_in)

In [ ]:
m=[]
m1=[]
i=0
for a in p:
    m.append(np.mean(a[1:]-a[:-1]))
    m1.append(a.size)
print(np.mean(m),np.mean(m1))

In [ ]:
# This cell is specifically made to run the averagedLOSv2() function because of more complex plotting details.

fig,ax=plt.subplots(nrows=3, ncols=1, sharex=True, sharey=True, figsize=(13,18),dpi=200)
fig.text(0.5, 0.0, 'Distance from Halo Center (R/R200b)', ha='center',fontsize=20)
fig.text(0.0, 0.5, 'Density Contrast', va='center', rotation='vertical',fontsize=20)

plt.subplot(3,1,1)
plt.plot(p_b1_r, df_b1_r, 'k', color='#A30000',label="b1")
plt.fill_between(p_b1_r, pt84_b1_r, pt16_b1_r,alpha=0.5, edgecolor='#FF6B6B', facecolor='#FFADAD')
plt.ylim([0.1,3e1])
plt.yscale('log')
plt.legend()

plt.subplot(3,1,2)
plt.plot(p_alpha_r, df_alpha_r, 'k', color='#00086B',label="Alpha")
plt.fill_between(p_alpha_r, pt84_alpha_r, pt16_alpha_r,alpha=0.5, edgecolor='#3a9cff', facecolor='#9bcdff')
plt.ylim([0.1,3e1])
plt.yscale('log')
plt.legend()

plt.subplot(3,1,3)
plt.plot(p_t10_r, df_t10_r, 'k', color='#036800',label="T10")
plt.fill_between(p_t10_r, pt84_t10_r, pt16_t10_r,alpha=0.5, edgecolor='#49c145', facecolor='#96ff93')
plt.ylim([0.1,3e1])
plt.yscale('log')
plt.legend()